# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read in the file from Part 1
data = pd.read_csv("output_data/cities.csv")
city_df = pd.DataFrame(data)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,okakarara,-20.5833,17.4333,70.00,58,77,2.44,NaN,1611873976
1,spokane,47.6166,-117.3677,42.80,61,90,1.61,US,1611873972
2,punta arenas,-53.1500,-70.9167,46.40,66,75,17.27,CL,1611873977
3,ismailia,30.6043,32.2723,53.06,42,59,14.50,EG,1611873889
4,nehe,48.4833,124.8333,-19.28,85,0,2.51,CN,1611873977


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
city_df_na = city_df.dropna()
# Store 'Lat' and 'Lng' into  locations 
locations = city_df_na[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
city_df_na = city_df.dropna()
humidity_per = city_df_na["Humidity"].astype(float)

In [4]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_per, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

#fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# ideal vacation conditions
temp_max = 90
temp_min = 80
humid_max = 50
cloud_max = 50
vacation_df = city_df_na.loc[(city_df_na["Max Temp"] <= temp_max) & (city_df_na["Max Temp"] >= temp_min) &
                            (city_df_na["Humidity"] <= humid_max) & (city_df_na["Cloudiness"] <= cloud_max)]
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
187,malakal,9.5334,31.6605,80.89,16,0,8.46,SS,1611874953
270,salto,-31.3833,-57.9667,87.01,50,27,8.01,UY,1611875332
530,alice springs,-23.7000,133.8833,86.00,42,0,10.36,AU,1611876024
584,guanare,9.0418,-69.7421,81.41,47,8,2.33,VE,1611876276


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_df.reset_index(drop=True)
# set up additional columns to hold information
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,malakal,9.5334,31.6605,80.89,16,0,8.46,SS,1611874953,
1,salto,-31.3833,-57.9667,87.01,50,27,8.01,UY,1611875332,
2,alice springs,-23.7000,133.8833,86.00,42,0,10.36,AU,1611876024,
3,guanare,9.0418,-69.7421,81.41,47,8,2.33,VE,1611876276,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
latp = hotel_df.loc[0,'Lat']
lngp = hotel_df.loc[0,'Lng']

coord = str(latp)+"," + str(lngp)

params = {
    "location": coord, 
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
}

# get restaurant type from df
    #restr_type = row['ethnicity']

    # add keyword to params dict
    #params['keyword'] = restr_type
    
response = requests.get(base_url, params=params).json()


# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 9.515723699999999,
                    "lng": 31.6559863
                },
                "viewport": {
                    "northeast": {
                        "lat": 9.5174038302915,
                        "lng": 31.6575305802915
                    },
                    "southwest": {
                        "lat": 9.514705869708495,
                        "lng": 31.6548326197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Nile Palace Hotel",
            "place_id": "ChIJ6TMQvZQy_RYRSWqKBv8Fl-c",
            "plus_code": {
                "compound_code": "GM84+79 Malakal, South Sudan",
                "global_code": "6GXHGM84+79"
            },
            "rating

In [42]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    latp = row['Lat']
    lngp = row['Lng']

    coord = str(latp)+"," + str(lngp)

    params = {
        "location": coord, 
        "radius": "5000",
        "type": "lodging",
        "key": g_key,
    }
    
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: malakal.
Closest hotel is Nile Palace Hotel.
------------
Retrieving Results for Index 1: salto.
Closest hotel is Gran Hotel Uruguay.
------------
Retrieving Results for Index 2: alice springs.
Closest hotel is Desert Palms Alice Springs.
------------
Retrieving Results for Index 3: guanare.
Closest hotel is Nuevo Hotel Coromoto.
------------


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Save the figure
plt.savefig("output_data/HeatMap.png")

<Figure size 432x288 with 0 Axes>